In [2]:
from binance.client import Client
import pandas as pd


In [3]:
# testing account keys
api_key = 'ZVEsEYIAPzAcPuFuhz0Jzm7Yk3Op6JDWSwIJxZmFYDF4mrrU9LJK4HtWcX7939VX'
secret_key = 'ooLajgeO1iBfl4SDVLbtAlloODxvvZ8d6MfhqK9c1pyBl3G7sfGg7DzMxTJapMev'
client = Client(api_key=api_key, api_secret=secret_key,
                tld='com', testnet=True)


In [4]:
client


In [5]:
client.get_system_status()


{'status': 0, 'msg': 'normal'}

In [6]:
account = client.get_account()
account


{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1659697517229,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [7]:
df = pd.DataFrame(account['balances'])
df


,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [8]:
df['free'] = pd.to_numeric(df['free'])
df['locked'] = pd.to_numeric(df['free'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   8 non-null      object 
 1   free    8 non-null      float64
 2   locked  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 320.0+ bytes


In [9]:
client.get_asset_balance('BTC')
client.get_asset_balance('USDT')
# get current prices for all pairs available
client.get_all_tickers()


[{'symbol': 'BNBBUSD', 'price': '266.10000000'},
 {'symbol': 'BTCBUSD', 'price': '18849.52000000'},
 {'symbol': 'ETHBUSD', 'price': '1577.67000000'},
 {'symbol': 'LTCBUSD', 'price': '55.21000000'},
 {'symbol': 'TRXBUSD', 'price': '0.06088000'},
 {'symbol': 'XRPBUSD', 'price': '0.32390000'},
 {'symbol': 'BNBUSDT', 'price': '266.10000000'},
 {'symbol': 'BTCUSDT', 'price': '18840.02000000'},
 {'symbol': 'ETHUSDT', 'price': '1577.74000000'},
 {'symbol': 'LTCUSDT', 'price': '55.18000000'},
 {'symbol': 'TRXUSDT', 'price': '0.06087000'},
 {'symbol': 'XRPUSDT', 'price': '0.32430000'},
 {'symbol': 'BNBBTC', 'price': '0.01411500'},
 {'symbol': 'ETHBTC', 'price': '0.08370700'},
 {'symbol': 'LTCBTC', 'price': '0.00292600'},
 {'symbol': 'TRXBTC', 'price': '0.00000323'},
 {'symbol': 'XRPBTC', 'price': '0.00001722'},
 {'symbol': 'LTCBNB', 'price': '0.20750000'},
 {'symbol': 'TRXBNB', 'price': '0.00022880'},
 {'symbol': 'XRPBNB', 'price': '0.00121500'}]

In [10]:
last24 = client.get_ticker(symbol='BTCUSDT')
last24


{'symbol': 'BTCUSDT',
 'priceChange': '-891.60000000',
 'priceChangePercent': '-4.519',
 'weightedAvgPrice': '19723.86014402',
 'prevClosePrice': '19732.07000000',
 'lastPrice': '18840.02000000',
 'lastQty': '0.00233200',
 'bidPrice': '18839.03000000',
 'bidQty': '0.14385100',
 'askPrice': '18839.89000000',
 'askQty': '0.03798300',
 'openPrice': '19731.62000000',
 'highPrice': '20354.52000000',
 'lowPrice': '6039.21000000',
 'volume': '9153.96576100',
 'quoteVolume': '180551540.43313691',
 'openTime': 1662407604295,
 'closeTime': 1662494004295,
 'firstId': 5306563,
 'lastId': 5484668,
 'count': 178106}

In [11]:
timestamp = client._get_earliest_valid_timestamp(
    symbol='BTCUSDT', interval='1h')
timestamp
# resets every month


1659657600000

In [12]:
pd.to_datetime(timestamp, unit='ms')


Timestamp('2022-08-05 00:00:00')

In [13]:
def get_history(symbol, interval, start, end=None):
    bars = client.get_historical_klines(
        symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(bars)
    df['Date'] = pd.to_datetime(df.iloc[:, 0], unit='ms')
    df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Clos Time', 'Quote Asset Volume',
                  'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore', 'Date']
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
    df.set_index('Date', inplace=True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df


In [14]:
df = get_history('BTCUSDT', '1d', timestamp)
df


,Open,High,Low,Close,Volume
Date,,,,,
2022-08-05,23671.60,25000.00,19182.70,23311.00,4188.570208
2022-08-06,23311.00,23454.23,10000.00,22953.15,6060.722177
2022-08-07,22953.99,23659.75,22751.36,23174.89,5921.778279
2022-08-08,23174.89,69314.66,22895.51,23813.42,7281.646347
2022-08-09,23812.30,24402.15,22549.05,23150.15,7583.533650
2022-08-10,23149.55,60000.00,22501.08,23953.93,7542.462312
2022-08-11,23954.42,36668.05,4670.88,23935.47,7637.853666
2022-08-12,23933.83,51000.00,9500.00,24399.68,7484.121732
2022-08-13,24398.19,50000.00,19598.80,24443.30,6776.329880


In [15]:
from binance import ThreadedWebsocketManager


In [16]:
def stream_data(msg):
    time = pd.to_datetime(msg['E'], unit='ms')
    price = msg['c']
    print('Time: {} | Price: {}'.format(time, price))


In [17]:
twm = ThreadedWebsocketManager()
twm.start()


In [18]:
twm.start_symbol_miniticker_socket(callback=stream_data, symbol='BTCUSDT')


'btcusdt@miniTicker'

In [19]:
twm.stop()


In [20]:
# buy 0.1 BTC
order = client.create_order(
    symbol='BTCUSDT', side='BUY', type='MARKET', quantity=0.1)


In [21]:
order

{'symbol': 'BTCUSDT',
 'orderId': 17738707,
 'orderListId': -1,
 'clientOrderId': 'ORmCIuTH6c8ts6KL06svGP',
 'transactTime': 1662494007018,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '1883.88597222',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '18838.82000000',
   'qty': '0.00265500',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5484671},
  {'price': '18838.86000000',
   'qty': '0.09342400',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5484672},
  {'price': '18838.88000000',
   'qty': '0.00392100',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5484673}]}

In [22]:
df = pd.DataFrame(order['fills'])
df

,price,qty,commission,commissionAsset,tradeId
0,18838.82000000,0.00265500,0.00000000,BTC,5484671
1,18838.86000000,0.09342400,0.00000000,BTC,5484672
2,18838.88000000,0.00392100,0.00000000,BTC,5484673


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            3 non-null      object
 1   qty              3 non-null      object
 2   commission       3 non-null      object
 3   commissionAsset  3 non-null      object
 4   tradeId          3 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 248.0+ bytes


In [24]:
num_columns = ['price','qty','commission']
for column in num_columns:
    df[column] = pd.to_numeric(df[column])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            3 non-null      float64
 1   qty              3 non-null      float64
 2   commission       3 non-null      float64
 3   commissionAsset  3 non-null      object 
 4   tradeId          3 non-null      int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 248.0+ bytes


In [25]:
df.qty.sum()

0.09999999999999999

In [26]:
df.price.mul(df.qty).sum()

1883.88597222

In [27]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1662494007018,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.10000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '8116.11402778', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [28]:
order = client.create_order(symbol='BTCUSDT',side='BUY',type='MARKET',quoteOrderQty=1000)

In [29]:
order

{'symbol': 'BTCUSDT',
 'orderId': 17738712,
 'orderListId': -1,
 'clientOrderId': 'sMlaOcD4eKscOSM5RstgWK',
 'transactTime': 1662494008142,
 'price': '0.00000000',
 'origQty': '0.05307900',
 'executedQty': '0.05307900',
 'cummulativeQuoteQty': '999.98235129',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '18839.51000000',
   'qty': '0.05307900',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5484674}]}

In [30]:
avg_price = float(order['cummulativeQuoteQty']) / float(order['origQty'])
avg_price

18839.51

In [31]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1662494008142,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.15307900', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '7116.13167649', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [32]:
# selling 0.13092100 BTC
order = client.create_order(symbol='BTCUSDT',side='SELL',type='MARKET',quantity=0.13092100)

In [33]:
order

{'symbol': 'BTCUSDT',
 'orderId': 17738717,
 'orderListId': -1,
 'clientOrderId': 'DiIP0W2tOhQhRhzpLJzYsd',
 'transactTime': 1662494008639,
 'price': '0.00000000',
 'origQty': '0.13092100',
 'executedQty': '0.13092100',
 'cummulativeQuoteQty': '2466.38117845',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '18838.70000000',
   'qty': '0.10449600',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 5484675},
  {'price': '18838.69000000',
   'qty': '0.02642500',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 5484676}]}

In [34]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1662494008639,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.02215800', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9582.51285494', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [35]:
order1 = client.create_order(symbol='BTCUSDT', side='BUY',
                             type='LIMIT', quantity=0.1, timeInForce="GTC", price=30000)


In [36]:
order1

{'symbol': 'BTCUSDT',
 'orderId': 17738718,
 'orderListId': -1,
 'clientOrderId': 'S8m8TEFvHbyjdLZmcEnOSj',
 'transactTime': 1662494009081,
 'price': '30000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '1883.95400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'fills': [{'price': '18839.54000000',
   'qty': '0.10000000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 5484677}]}

In [37]:
order_id_buy = order1['orderId']
order_id_buy

17738718

In [38]:
client.get_order(symbol='BTCUSDT',orderId=order_id_buy)

{'symbol': 'BTCUSDT',
 'orderId': 17738718,
 'orderListId': -1,
 'clientOrderId': 'S8m8TEFvHbyjdLZmcEnOSj',
 'price': '30000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '1883.95400000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1662494009081,
 'updateTime': 1662494009081,
 'isWorking': True,
 'origQuoteOrderQty': '0.00000000'}

In [39]:
order2 = client.create_order(symbol='BTCUSDT', side='SELL',
                             type='LIMIT', quantity=0.1, timeInForce="GTC", price=60000)


In [40]:
order2

{'symbol': 'BTCUSDT',
 'orderId': 17738723,
 'orderListId': -1,
 'clientOrderId': 'AbYUByp9PY9TyVIhmP1j7F',
 'transactTime': 1662494009581,
 'price': '60000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'fills': []}

In [41]:
order_id_sell = order2['orderId']
order_id_sell

17738723

In [42]:
open_orders = client.get_open_orders(symbol='BTCUSDT')
open_orders

[{'symbol': 'BTCUSDT',
  'orderId': 17738723,
  'orderListId': -1,
  'clientOrderId': 'AbYUByp9PY9TyVIhmP1j7F',
  'price': '60000.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1662494009581,
  'updateTime': 1662494009581,
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'}]

In [43]:
pd.DataFrame(open_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty
0,BTCUSDT,17738723,-1,AbYUByp9PY9TyVIhmP1j7F,60000.00000000,0.10000000,0.00000000,0.00000000,NEW,GTC,LIMIT,SELL,0.00000000,0.00000000,1662494009581,1662494009581,True,0.00000000


In [44]:
result = client.cancel_order(symbol='BTCUSDT',orderId=order_id_buy)
result

BinanceAPIException: APIError(code=-2011): Unknown order sent.

In [ ]:
result = client.cancel_order(symbol='BTCUSDT',orderId=order_id_sell)
result

{'symbol': 'BTCUSDT',
 'origClientOrderId': 'SXFNoRI6LgMUEAMyd25BBj',
 'orderId': 2465462,
 'orderListId': -1,
 'clientOrderId': 'INDwo7wwqIZF7AD5uvs7b6',
 'price': '60000.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL'}

In [ ]:
all_orders = client.get_all_orders(symbol='BTCUSDT')
all_orders

11

In [ ]:
pd.DataFrame(all_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty
0,BTCUSDT,2319659,-1,pokYC1XgSnbv79IHQ9SpPk,0.00000000,0.10000000,0.10000000,3215.07900592,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1652110311531,1652110311531,True,0.00000000
1,BTCUSDT,2323660,-1,QaKtjWOGyZfqNkicQ2Fcv1,0.00000000,0.03092100,0.03092100,999.97369923,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1652110969820,1652110969820,True,1000.00000000
2,BTCUSDT,2325247,-1,E7mb4eKWuHMURn9EkggsyY,0.00000000,0.13092100,0.13092100,4232.23912888,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1652111235049,1652111235049,True,0.00000000
3,BTCUSDT,2455333,-1,F6QhrkG75EK3W7UtXufGWT,30000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,BUY,0.00000000,0.00000000,1652131557823,1652133177114,True,0.00000000
4,BTCUSDT,2464244,-1,VG9Yyvg0ELXPAJNa86kWzg,60000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1652133032799,1652133206339,True,0.00000000
5,BTCUSDT,2464286,-1,L22tBtF2dEbg7TdYqCSmDn,60000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1652133039976,1652133187897,True,0.00000000
6,BTCUSDT,2465449,-1,MYeaGTjB6QCeiRwRvkG0hW,0.00000000,0.10000000,0.10000000,3127.48210372,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1652133231632,1652133231632,True,0.00000000
7,BTCUSDT,2465454,-1,UqynH2uwU88bIef8Ugruth,0.00000000,0.03197600,0.03197600,999.99568032,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1652133232367,1652133232367,True,1000.00000000
8,BTCUSDT,2465455,-1,XhcnUU3dJb6uJCF01XRoyb,0.00000000,0.13092100,0.13092100,4094.18131750,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1652133232883,1652133232883,True,0.00000000
9,BTCUSDT,2465460,-1,acMJauBTqv4tlGQXnw3itk,30000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,BUY,0.00000000,0.00000000,1652133233349,1652133234661,True,0.00000000


In [ ]:
trades = client.get_my_trades(symbol='BTCUSDT')
trades

[{'symbol': 'BTCUSDT',
  'id': 927128,
  'orderId': 2319659,
  'orderListId': -1,
  'price': '32148.78000000',
  'qty': '0.05741400',
  'quoteQty': '1845.79005492',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1652110311531,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 927129,
  'orderId': 2319659,
  'orderListId': -1,
  'price': '32153.50000000',
  'qty': '0.04258600',
  'quoteQty': '1369.28895100',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1652110311531,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 929301,
  'orderId': 2323660,
  'orderListId': -1,
  'price': '32339.63000000',
  'qty': '0.03092100',
  'quoteQty': '999.97369923',
  'commission': '0.00000000',
  'commissionAsset': 'BTC',
  'time': 1652110969820,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 930187,
  'orderId': 2325247,
  'orde

In [ ]:
df = pd.DataFrame(trades)
df

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,927128,2319659,-1,32148.78000000,0.05741400,1845.79005492,0.00000000,BTC,1652110311531,True,False,True
1,BTCUSDT,927129,2319659,-1,32153.50000000,0.04258600,1369.28895100,0.00000000,BTC,1652110311531,True,False,True
2,BTCUSDT,929301,2323660,-1,32339.63000000,0.03092100,999.97369923,0.00000000,BTC,1652110969820,True,False,True
3,BTCUSDT,930187,2325247,-1,32326.68000000,0.07733600,2500.01612448,0.00000000,USDT,1652111235049,False,False,True
4,BTCUSDT,930188,2325247,-1,32326.64000000,0.05358500,1732.22300440,0.00000000,USDT,1652111235049,False,False,True
5,BTCUSDT,1001944,2465449,-1,31273.32000000,0.02295600,717.91033392,0.00000000,BTC,1652133231632,True,False,True
6,BTCUSDT,1001945,2465449,-1,31274.05000000,0.01000000,312.74050000,0.00000000,BTC,1652133231632,True,False,True
7,BTCUSDT,1001946,2465449,-1,31275.45000000,0.06704400,2096.83126980,0.00000000,BTC,1652133231632,True,False,True
8,BTCUSDT,1001947,2465454,-1,31273.32000000,0.03197600,999.99568032,0.00000000,BTC,1652133232367,True,False,True
9,BTCUSDT,1001948,2465455,-1,31273.31000000,0.08122000,2540.01823820,0.00000000,USDT,1652133232883,False,False,True


In [ ]:
df.time = pd.to_datetime(df.time,unit='ms')
df

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,927128,2319659,-1,32148.78000000,0.05741400,1845.79005492,0.00000000,BTC,2022-05-09 15:31:51.531,True,False,True
1,BTCUSDT,927129,2319659,-1,32153.50000000,0.04258600,1369.28895100,0.00000000,BTC,2022-05-09 15:31:51.531,True,False,True
2,BTCUSDT,929301,2323660,-1,32339.63000000,0.03092100,999.97369923,0.00000000,BTC,2022-05-09 15:42:49.820,True,False,True
3,BTCUSDT,930187,2325247,-1,32326.68000000,0.07733600,2500.01612448,0.00000000,USDT,2022-05-09 15:47:15.049,False,False,True
4,BTCUSDT,930188,2325247,-1,32326.64000000,0.05358500,1732.22300440,0.00000000,USDT,2022-05-09 15:47:15.049,False,False,True
5,BTCUSDT,1001944,2465449,-1,31273.32000000,0.02295600,717.91033392,0.00000000,BTC,2022-05-09 21:53:51.632,True,False,True
6,BTCUSDT,1001945,2465449,-1,31274.05000000,0.01000000,312.74050000,0.00000000,BTC,2022-05-09 21:53:51.632,True,False,True
7,BTCUSDT,1001946,2465449,-1,31275.45000000,0.06704400,2096.83126980,0.00000000,BTC,2022-05-09 21:53:51.632,True,False,True
8,BTCUSDT,1001947,2465454,-1,31273.32000000,0.03197600,999.99568032,0.00000000,BTC,2022-05-09 21:53:52.367,True,False,True
9,BTCUSDT,1001948,2465455,-1,31273.31000000,0.08122000,2540.01823820,0.00000000,USDT,2022-05-09 21:53:52.883,False,False,True
